# Importing the necessary libraries to import, merge and manipulate the data

In [1]:
import os
import pandas as pd

In [62]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = r"C:\Users\Gwenn\Desktop\Data Material\Sales_Data"
files = [file for file in os.listdir(path) if not file.startswith('.')] # Ignore hidden files

all_months_data = pd.DataFrame()

for file in files:
    current_data = pd.read_csv(path+"/"+file)
    all_months_data = pd.concat([all_months_data, current_data])
    
all_months_data.to_csv("all_data_copy.csv", index=False)

# Importing the Pyspark libraries 

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [5]:
spark=SparkSession.builder.getOrCreate()

# Uploading the merged files into a Spark dataframe

In [6]:
df=spark.read.csv(r'C:\Users\Gwenn\all_data_copy.csv',header=True,
    inferSchema=True)

In [7]:
df.show()

+--------+--------------------+----------------+----------+----------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+----------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|  04/19/19 08:46|917 1st St, Dalla...|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/2019 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|04/12/2019 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|04/12/2019 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99|  04/30/19 09:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|  04/29/19 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|               1|     99.99|04/02/2019 07:46|668 Center St, Se...|
|  176564|USB-C Char

# Inspecting the data types

In [8]:
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



# Dropping all null values

In [9]:
df=df.dropna(how='any')

# Calculating the total sold items for each product

In [10]:
df.groupby('Product').agg(F.sum('Quantity Ordered').alias('Total Sold Items')).orderBy('Total Sold Items',ascending=False).show()

+--------------------+----------------+
|             Product|Total Sold Items|
+--------------------+----------------+
|AAA Batteries (4-...|           31017|
|AA Batteries (4-p...|           27635|
|USB-C Charging Cable|           23975|
|Lightning Chargin...|           23217|
|    Wired Headphones|           20557|
|Apple Airpods Hea...|           15661|
|Bose SoundSport H...|           13457|
|    27in FHD Monitor|            7550|
|              iPhone|            6849|
|27in 4K Gaming Mo...|            6244|
|34in Ultrawide Mo...|            6199|
|        Google Phone|            5532|
|       Flatscreen TV|            4819|
|  Macbook Pro Laptop|            4728|
|     ThinkPad Laptop|            4130|
|        20in Monitor|            4129|
|     Vareebadd Phone|            2068|
|  LG Washing Machine|             666|
|            LG Dryer|             646|
+--------------------+----------------+



# Converting the Price Each column into a float

In [11]:
df=df.withColumn('Price Each',df['Price Each'].cast('float'))

In [12]:
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: float (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



# Multiplying Price by Quantity to get Total Sales

In [13]:
from pyspark.sql.functions import col

In [14]:
df=df.withColumn('Total Sales',col('Quantity Ordered') * col('Price Each'))

In [15]:
df.show()

+--------+--------------------+----------------+----------+----------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Total Sales|
+--------+--------------------+----------------+----------+----------------+--------------------+-----------+
|  176558|USB-C Charging Cable|               2|     11.95|  04/19/19 08:46|917 1st St, Dalla...|       23.9|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/2019 22:30|682 Chestnut St, ...|      99.99|
|  176560|        Google Phone|               1|     600.0|04/12/2019 14:38|669 Spruce St, Lo...|      600.0|
|  176560|    Wired Headphones|               1|     11.99|04/12/2019 14:38|669 Spruce St, Lo...|      11.99|
|  176561|    Wired Headphones|               1|     11.99|  04/30/19 09:27|333 8th St, Los A...|      11.99|
|  176562|USB-C Charging Cable|               1|     11.95|  04/29/19 13:03|381 Wilson St, Sa...|      11.95|
|  176563|

# Grouping by Products to see which ones grosses in the most money

In [16]:
total_sales=df.groupby('Product').agg(F.sum('Total Sales').alias('Total Product Sales'),
                         F.sum('Quantity Ordered').alias('Total Quantity Sold')).orderBy('Total Product Sales',ascending=False)

In [17]:
total_sales.show()

+--------------------+-------------------+-------------------+
|             Product|Total Product Sales|Total Quantity Sold|
+--------------------+-------------------+-------------------+
|  Macbook Pro Laptop|          8037600.0|               4728|
|              iPhone|          4794300.0|               6849|
|     ThinkPad Laptop| 4129958.6596679688|               4130|
|        Google Phone|          3319200.0|               5532|
|27in 4K Gaming Mo...| 2435097.4990234375|               6244|
|34in Ultrawide Mo...| 2355557.9494628906|               6199|
|Apple Airpods Hea...|          2349150.0|              15661|
|       Flatscreen TV|          1445700.0|               4819|
|Bose SoundSport H...| 1345565.4012680054|              13457|
|    27in FHD Monitor| 1132424.5414733887|               7550|
|     Vareebadd Phone|           827200.0|               2068|
|        20in Monitor|  454148.7011795044|               4129|
|  LG Washing Machine|           399600.0|             

# Finding out the average amount spent per purchase

In [21]:
total_sales_dollars=total_sales.agg(F.sum('Total Product Sales')).collect()[0][0]

In [22]:
total_sold_products=total_sales.agg(F.sum('Total Quantity Sold')).collect()[0][0]

In [23]:
print('The average amount spent per purchase is $',total_sales_dollars/total_sold_products)

The average amount spent per purchase is $ 164.97130652214278


# Finding out how much percentage each item accounts for 

In [28]:
total_sales=total_sales.withColumn('Percentage Total Product Sold',(col('Total Quantity Sold')/total_sold_products)*100)

In [29]:
total_sales.show()

+--------------------+-------------------+-------------------+-----------------------------+
|             Product|Total Product Sales|Total Quantity Sold|Percentage Total Product Sold|
+--------------------+-------------------+-------------------+-----------------------------+
|  Macbook Pro Laptop|          8037600.0|               4728|            2.261346189717762|
|              iPhone|          4794300.0|               6849|           3.2757952735568847|
|     ThinkPad Laptop| 4129958.6596679688|               4130|            1.975329899224695|
|        Google Phone|          3319200.0|               5532|            2.645889831116468|
|27in 4K Gaming Mo...| 2435097.4990234375|               6244|            2.986430966285471|
|34in Ultrawide Mo...| 2355557.9494628906|               6199|           2.9649080012818123|
|Apple Airpods Hea...|          2349150.0|              15661|            7.490470109384491|
|       Flatscreen TV|          1445700.0|               4819|        

In [32]:
split_col=pyspark.sql.functions.split(df['Order Date'],'')

In [33]:
df.show()

+--------+--------------------+----------------+----------+----------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Total Sales|
+--------+--------------------+----------------+----------+----------------+--------------------+-----------+
|  176558|USB-C Charging Cable|               2|     11.95|  04/19/19 08:46|917 1st St, Dalla...|       23.9|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/2019 22:30|682 Chestnut St, ...|      99.99|
|  176560|        Google Phone|               1|     600.0|04/12/2019 14:38|669 Spruce St, Lo...|      600.0|
|  176560|    Wired Headphones|               1|     11.99|04/12/2019 14:38|669 Spruce St, Lo...|      11.99|
|  176561|    Wired Headphones|               1|     11.99|  04/30/19 09:27|333 8th St, Los A...|      11.99|
|  176562|USB-C Charging Cable|               1|     11.95|  04/29/19 13:03|381 Wilson St, Sa...|      11.95|
|  176563|

In [35]:
df=df.withColumn('Order Date Updated',split_col.getItem(0))

In [36]:
df=df.withColumn('Order Time',split_col.getItem(1))

In [37]:
split_col=pyspark.sql.functions.split(df['Order Time'],':')

In [38]:
df=df.withColumn('Order Hour',split_col.getItem(0))

In [39]:
df.show()

+--------+--------------------+----------------+----------+----------------+--------------------+-----------+------------------+----------+----------+
|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Total Sales|Order Date Updated|Order Time|Order Hour|
+--------+--------------------+----------------+----------+----------------+--------------------+-----------+------------------+----------+----------+
|  176558|USB-C Charging Cable|               2|     11.95|  04/19/19 08:46|917 1st St, Dalla...|       23.9|                 0|         4|         4|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/2019 22:30|682 Chestnut St, ...|      99.99|                 0|         4|         4|
|  176560|        Google Phone|               1|     600.0|04/12/2019 14:38|669 Spruce St, Lo...|      600.0|                 0|         4|         4|
|  176560|    Wired Headphones|               1|     11.99|04/12/2019 14:38|669 Spruce St, Lo.

In [59]:
orders_by_hours=df.groupby('Order Hour').agg(F.sum('Total Sales').alias('Total Hourly Sales'),
                            F.sum('Quantity Ordered').alias('Hourly Quantity Ordered')).orderBy('Total Hourly Sales',ascending=False)

In [48]:
split_col=pyspark.sql.functions.split(df['Order Date'],'/')

In [51]:
df=df.withColumn('Order Day',split_col.getItem(1))

In [57]:
total_sales_per_day=df.groupby('Order Day').agg(F.sum('Total Sales').alias('Total Sales'),
                           F.sum('Quantity Ordered').alias('Total Sold Products')).orderBy('Total Sales',ascending=False)

In [58]:
total_sales_per_day.show()

+---------+------------------+-------------------+
|Order Day|       Total Sales|Total Sold Products|
+---------+------------------+-------------------+
|       09| 1169250.424434185|               6836|
|       10| 1169108.723827362|               6965|
|       25|1168818.5841395855|               6804|
|       11|1167958.4840314388|               6760|
|       01|1166319.2442250252|               6867|
|       18|1164031.5440473557|               6846|
|       04| 1163857.714392662|               6813|
|       06| 1151956.084501028|               6778|
|       20|1144375.4642179012|               6923|
|       26|1140443.2842223644|               7018|
|       22|1139919.2846345901|               6887|
|       17|1139307.9843757153|               6972|
|       02| 1137457.034198761|               6962|
|       13|1137392.5339622498|               7112|
|       05|1135284.7244741917|               6816|
|       14|1133909.6541388035|               6907|
|       27|1128343.1839623451| 

# Cleaning the "Purchase Address" column to extract some insights about States and Cities

In [41]:
df.select('Purchase Address').show(truncate=False)

+----------------------------------------+
|Purchase Address                        |
+----------------------------------------+
|917 1st St, Dallas, TX 75001            |
|682 Chestnut St, Boston, MA 02215       |
|669 Spruce St, Los Angeles, CA 90001    |
|669 Spruce St, Los Angeles, CA 90001    |
|333 8th St, Los Angeles, CA 90001       |
|381 Wilson St, San Francisco, CA 94016  |
|668 Center St, Seattle, WA 98101        |
|790 Ridge St, Atlanta, GA 30301         |
|915 Willow St, San Francisco, CA 94016  |
|83 7th St, Boston, MA 02215             |
|444 7th St, Los Angeles, CA 90001       |
|438 Elm St, Seattle, WA 98101           |
|657 Hill St, Dallas, TX 75001           |
|186 12th St, Dallas, TX 75001           |
|253 Johnson St, Atlanta, GA 30301       |
|149 Dogwood St, New York City, NY 10001 |
|214 Chestnut St, San Francisco, CA 94016|
|20 Hill St, Los Angeles, CA 90001       |
|20 Hill St, Los Angeles, CA 90001       |
|433 Hill St, New York City, NY 10001    |
+----------

In [42]:
split_col=pyspark.sql.functions.split(df['Purchase Address'],',')

In [43]:
#Getting the States
df=df.withColumn('State',split_col.getItem(2))

In [44]:
#Getting the City
df=df.withColumn('City',split_col.getItem(1))

In [45]:
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: float (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Total Sales: float (nullable = true)
 |-- Order Date Updated: string (nullable = true)
 |-- Order Time: string (nullable = true)
 |-- Order Hour: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)



# Finding out which states spend the most

In [46]:
df.groupby('City').agg(F.sum('Total Sales').alias('Total Sales by State')).orderBy('Total Sales by State',ascending=False).show()

+--------------+--------------------+
|          City|Total Sales by State|
+--------------+--------------------+
| San Francisco|   8262203.869155407|
|   Los Angeles|   5452570.772869825|
| New York City|   4664317.406611681|
|        Boston|  3661641.9913027287|
|       Atlanta|   2795498.565934658|
|        Dallas|   2767975.385901928|
|       Seattle|   2747755.465265751|
|      Portland|  2320490.5985319614|
|        Austin|  1819581.7407691479|
+--------------+--------------------+

